In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Add project root to path
sys.path.append(r"D:\data-problem-solving")

from db import load_table, engine

df =  load_table('su_server_utilization')

In [3]:
df["status_tim"] = pd.to_datetime(df["status_tim"])

# Sort by server and timestamp
df = df.sort_values(["server_id", "status_tim"])

# Shift previous event info
df["prev_time"] = df.groupby("server_id")["status_tim"].shift(1)
df["prev_status"] = df.groupby("server_id")["session_status"].shift(1)

# Calculate diff only for stop rows where previous was start
df["uptime_days"] = (df["status_tim"] - df["prev_time"]).dt.days
df.loc[~((df["session_status"] == "stop") & (df["prev_status"] == "start")), "uptime_days"] = 0

# Total uptime
total_uptime = df["uptime_days"].sum()

print("Total uptime (days):", total_uptime)

Total uptime (days): 9.0
